Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

In [1]:
import numpy as np
import pandas as pd
import sqlite3 
import pandasql 
import sqlalchemy
from sqlalchemy import orm  

In [17]:
adult_data= pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None,index_col=False)
lst_col=['AGE', 
        'WORKCLASS', 
        'FNLWGT', 
        'EDUCATION', 
        'EDUCATION_NUM', 
        'MARITAL_STATUS', 
        'OCCUPATION', 
        'RELATIONSHIP', 
        'RACE', 
        'SEX', 
        'CAPITAL_GAIN', 
        'CAPITAL_LOSS', 
        'HOURS_PER_WEEK', 
        'NATIVE_COUNTRY', 
        'GT50_OR_LT50K']

adult_data.columns=lst_col
adult_data.head()

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [13]:
engine = sqlalchemy.create_engine('sqlite:///:memory:',echo=False)
conn = engine.connect()

engine.execute('''
    CREATE TABLE IF NOT EXISTS ADULTS (
         AGE             INTEGER,
         WORKCLASS       VARCHAR(100),
         FNLWGT          INTEGER,
         EDUCATION       VARCHAR(100),
         EDUCATION_NUM   INTEGER,         
         MARITAL_STATUS  VARCHAR(100),         
         OCCUPATION      VARCHAR(100),
         RELATIONSHIP    VARCHAR(100),
         RACE            VARCHAR(100),
         SEX             VARCHAR(20),
         CAPITAL_GAIN    INTEGER,
         CAPITAL_LOSS    INTEGER,
         HOURS_PER_WEEK  INTEGER,
         NATIVE_COUNTRY  VARCHAR(100),
         GT50_OR_LT50K   VARCHAR(20))
''')

adult_sample= adult_data
dict_adult = adult_sample.to_dict(orient='records')

metadata = sqlalchemy.schema.MetaData(bind=engine)
ADULT_TAB = sqlalchemy.Table('ADULTS', metadata, autoload=True)


Session = orm.sessionmaker(bind=engine)
session = Session()
conn.execute(ADULT_TAB.insert(), dict_adult)
session.commit()
session.close()

sql_select="SELECT * FROM ADULTS LIMIT 5;"
conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 
result_adult_data

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [14]:
sql_update1 = " UPDATE ADULTS SET MARITAL_STATUS='Married' where MARITAL_STATUS like ' Married%';"
engine.execute(sql_update1)

sql_update2 = " UPDATE ADULTS SET NATIVE_COUNTRY='Not Known' where NATIVE_COUNTRY =' ?';"
engine.execute(sql_update2)

sql_select="SELECT * FROM ADULTS WHERE (NATIVE_COUNTRY='Not Known') OR (MARITAL_STATUS='Married') LIMIT 10;"
conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 
result_adult_data

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,53,Private,234721,11th,7,Married,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
2,28,Private,338409,Bachelors,13,Married,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
3,37,Private,284582,Masters,14,Married,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
4,49,Private,160187,9th,5,Married,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
5,52,Self-emp-not-inc,209642,HS-grad,9,Married,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
6,42,Private,159449,Bachelors,13,Married,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
7,37,Private,280464,Some-college,10,Married,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K
8,30,State-gov,141297,Bachelors,13,Married,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,India,>50K
9,40,Private,121772,Assoc-voc,11,Married,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,40,Not Known,>50K


In [18]:
## For Marital_Status

sql_delete= "DELETE FROM ADULTS WHERE MARITAL_STATUS =' ?' ;"
engine.execute(sql_delete)

sql_select="SELECT * FROM ADULTS WHERE MARITAL_STATUS =' ?' ;"
conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 
result_adult_data

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K


In [19]:
## For OCCUPATION
sql_delete= "DELETE FROM ADULTS WHERE OCCUPATION =' ?' ;"
engine.execute(sql_delete)

sql_select="SELECT * FROM ADULTS WHERE OCCUPATION =' ?' ;"
conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 
result_adult_data

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K


In [38]:
sql_select="SELECT * FROM ADULTS where AGE <=18 AND SEX=' Male';"
conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 
result_adult_data.head()

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,18,Private,446839,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,0,30,United-States,<=50K
1,18,Private,25828,11th,7,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,16,United-States,<=50K
2,18,Private,183930,HS-grad,9,Never-married,Other-service,Own-child,White,Male,0,0,12,United-States,<=50K
3,17,Private,245918,11th,7,Never-married,Other-service,Own-child,White,Male,0,0,12,United-States,<=50K
4,17,Private,191260,9th,5,Never-married,Other-service,Own-child,White,Male,1055,0,24,United-States,<=50K


In [45]:
sql_select="SELECT * FROM ADULTS"
sql_select=sql_select+ " WHERE SEX=' Female' "
sql_select=sql_select+ " AND MARITAL_STATUS=' Never-married'"

conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 
result_adult_data.head()

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
1,23,Private,122272,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,<=50K
2,18,Private,226956,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,30,Not Known,<=50K
3,18,Private,309634,11th,7,Never-married,Other-service,Own-child,White,Female,0,0,22,United-States,<=50K
4,43,Federal-gov,410867,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,50,United-States,>50K


In [ ]:
Task##5. Write two function queries

In [55]:
sql_select="SELECT MARITAL_STATUS, COUNT(*) AS COUNT_MARRIED FROM ADULTS GROUP BY MARITAL_STATUS;"
conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 
result_adult_data



,MARITAL_STATUS,COUNT_MARRIED
0,Divorced,4258
1,Never-married,9912
2,Separated,959
3,Widowed,840
4,Married,14749


In [46]:
sql_select="SELECT SEX, COUNT(*) AS COUNT_GENDER FROM ADULTS GROUP BY SEX;"
conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 
result_adult_data

,SEX,COUNT_GENDER
0,Female,9930
1,Male,20788
